### Advent Of Code Day 16: Packet Decoder

In [3]:
inputCode = "0052E4A00905271049796FB8872A0D25B9FB746893847236200B4F0BCE5194401C9B9E3F9C63992C8931A65A1CCC0D222100511A00BCBA647D98BE29A397005E55064A9DFEEC86600BD002AF2343A91A1CCE773C26600D126B69D15A6793BFCE2775D9E4A9002AB86339B5F9AB411A15CCAF10055B3EFFC00BCCE730112FA6620076268CE5CDA1FCEB69005A3800D24F4DB66E53F074F811802729733E0040E5C5E5C5C8015F9613937B83F23B278724068018014A00588014005519801EC04B220116CC0402000EAEC03519801A402B30801A802138801400170A0046A800C10001AB37FD8EB805D1C266963E95A4D1A5FF9719FEF7FDB4FB2DB29008CD2BAFA3D005CD31EB4EF2EBE4F4235DF78C66009E80293AE9310D3FCBFBCA440144580273BAEE17E55B66508803C2E0087E630F72BCD5E71B32CCFBBE2800017A2C2803D272BCBCD12BD599BC874B939004B5400964AE84A6C1E7538004CD300623AC6C882600E4328F710CC01C82D1B228980292ECD600B48E0526E506F700760CCC468012E68402324F9668028200C41E8A30E00010D8B11E62F98029801AB88039116344340004323EC48873233E72A36402504CB75006EA00084C7B895198001098D91AE2190065933AA6EB41AD0042626A93135681A400804CB54C0318032200E47B8F71C0001098810D61D8002111B228468000E5269324AD1ECF7C519B86309F35A46200A1660A280150968A4CB45365A03F3DDBAE980233407E00A80021719A1B4181006E1547D87C6008E0043337EC434C32BDE487A4AE08800D34BC3DEA974F35C20100BE723F1197F59E662FDB45824AA1D2DDCDFA2D29EBB69005072E5F2EDF3C0B244F30E0600AE00203229D229B342CC007EC95F5D6E200202615D000FB92CE7A7A402354EE0DAC0141007E20C5E87A200F4318EB0C"
len(inputCode)

1332

**Part One**

In [4]:
# Parse the hexadecimal packet and convert it to binary
def hexToBin(hexCode, lenght): return bin(int(hexCode[:lenght], 16))[2:].zfill(lenght * 4)

print("First three hex values converted to binary:", hexToBin(inputCode, 3))
print("First four hex values converted to binary: ", hexToBin(inputCode, 4))

inputCodeBin = hexToBin(inputCode, len(inputCode))

First three hex values converted to binary: 000000000101
First four hex values converted to binary:  0000000001010010


In [5]:
def readHeader(header):
    """ Returns both the packet Version and the type ID, from a 6 bits lenght header"""
    version, id = 0, 0
    for i, value in enumerate(header[:6]):
        if i < 3: version += 2 ** (2 - i) * int(value)
        else: id += 2 ** (5 - i) * int(value)
    return version, id

print(inputCodeBin[14:20], readHeader(inputCodeBin[14:20]))
    

101110 (5, 6)


In [6]:
# Read the bin code and return its value (used for getting packet values like version, id, etc)
def binToInt(binCode, lenght): return sum(2 ** i * int(value) for i, value in enumerate(binCode[:lenght][::-1]))

print("Value of first three bits: ", binToInt(inputCodeBin, 3))
print("Value of next three bits: ", binToInt(inputCodeBin[3:], 3))
print("Value of next five bits: ", binToInt(inputCodeBin[6:], 5))

Value of first three bits:  0
Value of next three bits:  0
Value of next five bits:  2


In [7]:
# Mask the binCode assuming packet encoding
def decodePacket(binCode):
    """ Return the packet and the lenght of the parsed packet. The packet is represented with
    a tuple (Version, ID, [values]), where [values] is either a list of subpackets (tuples) 
    contained in the outermost packet or a single literal value (ID == 4). 

    This function is recursively applied for getting all the subpackets information. """

    # Get the version and the id of the packet
    version, id = binToInt(binCode, 3), binToInt(binCode[3:], 3)
    ptrIndx = 6

    packetValues = []

    if id == 4: # Packet Literal Value
        
        value = 0

        # Keep reading packet groups of length 4 until last group
        while int(binCode[ptrIndx:ptrIndx+1]): 
            value = 16 * value + binToInt(binCode[ptrIndx+1:], 4)
            ptrIndx += 5

        # Read last packetGroup
        value = 16 * value + binToInt(binCode[ptrIndx+1:], 4)
        ptrIndx += 5

        # Get rid of leading zeros
        #ptrIndx += (4 - ptrIndx) % 4
        
        return (version, id, value), ptrIndx

    else: # Packet Operator Value (outermost)

        # Read Length Type ID
        lengthID = int(binCode[ptrIndx:ptrIndx+1])
        ptrIndx += 1

        if lengthID == 0: # 15 bits of total packet length
            
            totalPacketLenght = binToInt(binCode[ptrIndx:], 15)
            ptrIndx += 15

            # Keep reading subpackets until totalPacketLenght is reached
            finalPtrIndex = ptrIndx + totalPacketLenght
            while ptrIndx < finalPtrIndex:
                nextPacket, nextPtrIndx = decodePacket(binCode[ptrIndx: ])
                packetValues.append(nextPacket)
                ptrIndx += nextPtrIndx

            # Check good parsing stop condition
            assert ptrIndx == finalPtrIndex

            return (version, id, packetValues), ptrIndx 

        if lengthID == 1: # 11 bits of number of subpackets
            
            numberOfSubPackets = binToInt(binCode[ptrIndx:], 11)
            ptrIndx += 11

            # Read the following number of subpackets
            for j in range(numberOfSubPackets):
                nextPacket, nextPtrIndx = decodePacket(binCode[ptrIndx: ])
                packetValues.append(nextPacket)
                ptrIndx += nextPtrIndx
            
            return (version, id, packetValues), ptrIndx 

    # Return (version, id, []) for not successful decoded packets
    return (version, id, packetValues), ptrIndx 

In [8]:
# Compute the version sum of a list of paquets
def versionSum(listOfPackets):
    """ Return the version sum of the list of packets. Can be recursively applied! """
    totalSum = 0
    for packet in listOfPackets: 
        totalSum += packet[0]
        if type(packet[2]) == list: totalSum += versionSum(packet[2])
    return totalSum

**Part 1 Examples**

In [9]:
# Literal Value Packet
example1 = "D2FE28"
example1Packet, example1Ptr = decodePacket(hexToBin(example1, len(example1)))
print(example1Packet, versionSum([example1Packet]))

(6, 4, 2021) 6


In [10]:
# 0 ID Operator Packet containing two subpackets 
example2 = "38006F45291200"
example2Packet, example2Ptr = decodePacket(hexToBin(example2, len(example2)))
print(example2Packet, versionSum([example2Packet]))

(1, 6, [(6, 4, 10), (2, 4, 20)]) 9


In [11]:
example3 = "EE00D40C823060"
example3Packet, example3Ptr = decodePacket(hexToBin(example3, len(example3)))
print(example3Packet, versionSum([example3Packet]))

(7, 3, [(2, 4, 1), (4, 4, 2), (1, 4, 3)]) 14


In [12]:
example4 = "8A004A801A8002F478"
example4Packet, example4Ptr = decodePacket(hexToBin(example4, len(example4)))
print(example4Packet, versionSum([example4Packet]))

(4, 2, [(1, 2, [(5, 2, [(6, 4, 15)])])]) 16


In [13]:
example5 = "620080001611562C8802118E34"
example5Packet, example5Ptr = decodePacket(hexToBin(example5, len(example5)))
print(example5Packet, versionSum([example5Packet]))

(3, 0, [(0, 0, [(0, 4, 10), (5, 4, 11)]), (1, 0, [(0, 4, 12), (3, 4, 13)])]) 12


In [14]:
example6 = "C0015000016115A2E0802F182340"
example6Packet, example6Ptr = decodePacket(hexToBin(example6, len(example6)))
print(example6Packet, versionSum([example6Packet]))

(6, 0, [(0, 0, [(0, 4, 10), (6, 4, 11)]), (4, 0, [(7, 4, 12), (0, 4, 13)])]) 23


In [15]:
example7 = "A0016C880162017C3686B18A3D4780"
example7Packet, example7Ptr = decodePacket(hexToBin(example7, len(example7)))
print(example7Packet, versionSum([example7Packet]))

(5, 0, [(1, 0, [(3, 0, [(7, 4, 6), (6, 4, 6), (5, 4, 12), (2, 4, 15), (2, 4, 15)])])]) 31


In [16]:
# Can I concatenate examples?
example6_7 = example6 + example7
decodePacket(hexToBin(example6_7, len(example6_7)))

((6, 0, [(0, 0, [(0, 4, 10), (6, 4, 11)]), (4, 0, [(7, 4, 12), (0, 4, 13)])]),
 106)

In [17]:
inputPacket, inputPtr = decodePacket(hexToBin(inputCode, len(inputCode)))
versionSum([inputPacket])

986

**Part 2:** Calculate the value that the transmission represents

In [18]:
def product(lst):
    p = 1
    for i in lst:  
        p *= i
    return p

def calculateExpression(packet):
    """ Return the calculated expression the list of packets represents. Can be recursively applied! """
    result = 0
    packetID = packet[1]

    if packetID == 4: return packet[2]                              # Literal value
    
    # If is an operator packet, get the value of the subpackets
    subpackets = [calculateExpression(subpacket) for subpacket in packet[2]]

    if packetID == 0: return sum(subpackets)                        # Sum
    if packetID == 1: return product(subpackets)                    # Product
    if packetID == 2: return min(subpackets)                        # Min
    if packetID == 3: return max(subpackets)                        # Max
    if packetID == 5: return int(subpackets[0] > subpackets[1])     # Greater Than
    if packetID == 6: return int(subpackets[0] < subpackets[1])     # Lesss Than
    if packetID == 7: return int(subpackets[0] == subpackets[1])    # Equal

In [19]:
calculateExpression(inputPacket)

18234816469452